## First Program

Remember kids game for guessing numbers. It is a simple game played by two people. One picks a secret number between 1 and 10 and the other has to guess the secret number. <br>
`
Is it 2? - No.
Is it 5? - No.
Is it 3? - No.
Is it 7? - No.
Is it 1? - Yes!
` <br>
This game works fine for a small range and quicly becomes frustrating when the range is increased to 100 or more. This is because there is no way to improve your guess. You are likely to a lot more.<br>
The game can be made more interesting by giving a `Higher` or `Lower` hint after every guess. <br>
`
Is it 2? - No. It's lower.
Is it 1? - Yes.
`
<br>
Let's say your secret number is `1`. Consider the following friend: <br>
`
Friend: Is it 2?
You: No. It's lower.
Friend: Is it 7?
`
<br>
I bet you wonder what's wrong with this friend. `7` is not lower than `2`. Still, guessing `2` again would be as insane as it is absurd. Why is that? <br>
Because we are expected to use `the domain knowledge` to improve our guesses. <br>

Similarly, when playing a card game,
- Inexperienced players build a mental map using the cards in their hands and those on the table.
- More experienced players also take advantage of their problem space knowledge and the entire deck of cards.
- Highly experienced card players take into consideration the probabilities.
- Professionals pay attention to the way their competitors play as well.
<br>


>**A genetic algorithm does not know what lower means. It has *no intelligence*. It does not learn** <br>
It will *make same mistakes every time*. It can only be as good as its programmer can get. <br>
**And yet it *can be used to find solutions that its programmer would struggle to find*.** How is this possible?

### Genes

- In the begining, the genetic algorithm needs a gene set to use for building guesses.
- Here, the gene set will be a generic set of letters.
- It also needs a target password to guess.

In [1]:
geneSet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!."
target = "Hello World!"

### Generate a guess

In [2]:
import random

def generate_parent(length):
    genes=[]
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    return ''.join(genes)

`random.sample` takes `sampleSize` values from the input without replacement. This means that there will be no duplicates in the generated parent unless `geneSet` contains duplicates, or length is greater than `len(geneSet)`. The implementation above can generate a long string with a small set of genes and uses as many unique genes as possible.

### Fitness

- The fitness value is the only feedback that guides GA to a solution.
- In this project, the fitness value is the total number of letters in the guess that match the letter in the same position of the password.

In [3]:
def get_fitness(guess):
    return sum(1 for expected, actual in zip(target,guess) if expected == actual)

### Mutate

- The engine needs a way to produce a new guess by mutating the current one.
- The following implementation converts the parent string to an array with `list(patent`, then replaces 1 letter in the array with the randomly selected one from the `geneSet`, and finally recombines the result into a string with `''.join(ChildGenes)`.

In [4]:
def mutate(parent):
    index = random.randrange(0, len(parent))
    childGenes = list(parent)
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate \
        if newGene == childGenes[index] \
        else newGene
    return ''.join(childGenes)

This implementation uses an alternate replacement if the randomly selected `newGene` is the same as the one it is supposed to replace, which can prevent a significant number of wasted guesses.

### Display

- It is important to monitor what is happening so that the engine can be stopped if it gets stuck.

In [5]:
import datetime
...
def display(guess):
    timeDiff = datetime.datetime.now() - startTime
    fitness = get_fitness(guess)
    print("{0}\t{1}\t{2}".format(guess, fitness, str(timeDiff)))

### Main

The main program begins by initializing `bestParent` to a random sequence of letters and calling the display function.

In [6]:
random.seed()
startTime = datetime.datetime.now()
bestParent = generate_parent(len(target))
bestFitness = get_fitness(bestParent)
display(bestParent)

ElZ WSpACXwv	0	0:00:00.000178


The final piece is the heart of the genetic engine - a loop that:
- generates a guess,
- requests the `fitness` for that guess,
- compares the `fitness` to that of the previous best guess, and 
- keeps the guess with the better fitness.
This cycle repeats untill a `stop condition` occurs, in this case when all the letters in the guess match those in the target.

In [7]:
while True:
    child = mutate(bestParent)
    childFitness = get_fitness(child)
    if bestFitness >= childFitness:
        continue
    display(child)
    if childFitness >= len(bestParent):
        break
    bestFitness = childFitness
    bestParent = child

ElZ WSWACXwv	1	0:00:05.212893
ElZ WSWArXwv	2	0:00:05.213300
ElZ oSWArXwv	3	0:00:05.213786
ElZ oSWArXdv	4	0:00:05.214265
ElZ oSWorXdv	5	0:00:05.214345
HlZ oSWorXdv	6	0:00:05.215281
Hll oSWorXdv	7	0:00:05.216446
Hel oSWorXdv	8	0:00:05.216544
Hel oSWorldv	9	0:00:05.216583
HelloSWorldv	10	0:00:05.217478
HelloSWorld!	11	0:00:05.219173
Hello World!	12	0:00:05.230206


## Extract a reusable engine

- Move the `mutate` and `generate_parent` functions to the new file and rename them to `_mutate` and `_generate_parent`. This is how protected functions are named in Python. Protected functions are only accessible to other fuctions in the same module. 

### Generate and Mutate